# Keyword-Extraction using BERT

Use BERT Token Classification Model to extract keyword tokens from a sentence.

## Prepare Dataset for BERT.

Convert Sem-Eval 2010 keyword recognition dataset to BIO format dataset.

In [1]:
import os
from nltk.tokenize import sent_tokenize
import abbreviation
from abbreviation import limits

In [2]:
train_path = "maui-semeval2010-train"

In [3]:
txt = sorted([f for f in os.listdir(train_path) if not f.endswith("-justTitle.txt") and not f.endswith(".key") and not f.endswith("-CrowdCountskey")])
key = sorted([f for f in os.listdir(train_path) if f.endswith(".key")])

In [4]:
filekey = dict()
for i, k in enumerate(txt):
    filekey[key[i]] = k

In [5]:
def convert(key):
    sentences = ""
    for line in open(train_path + "/" + filekey[key], 'r'):
        sentences += (" " + line.rstrip())
    tokens = sent_tokenize(sentences)
    key_file = open(train_path + "/" + str(key),'r')
    keys = [line.strip() for line in key_file]
    key_sent = []
    labels = []

    for token in tokens:
        
        for word in token.split():
            
            if word.lower() in limits:
                token = token.replace(word, limits[word.lower()])
             
        punctuations = '''!()-[]{};:\,<>'"./?@#$%^&*_~'''

 

        no_punct = ""
        for char in token:
            if char not in punctuations:
                no_punct = no_punct + char
        

        
        token=no_punct
       
        z = ['O'] * len(token.split())
        for k in keys:

            if k in token.lower():

                if len(k.split())==1:
                    try:
                        
                        z[token.lower().split().index(k.lower().split()[0])] = 'B'

                    except ValueError:
                        continue
                        
                    
                elif len(k.split())>1:

                    try:
                        h = k.split()
                        def index(arr, num):
                            a = []
                            for i, x in enumerate(arr):
                                if x == num:
                                    a.append(i)
                            return(a)

                     
                            
                            
                        lis1 = index(token.lower().split(),k.lower().split()[0])
                        lis2 =  index(token.lower().split(),k.lower().split()[1])
                        x = []
                        for i in lis1:
                            for j in lis2:
                                if abs(i - j) == 1:
                                    x.append(i)
                        num = x[0]  
                        z[num] = 'B'
                        
                        for j in range(1, len(h)):
                            z[num + j] = 'I'
                            
                    except:
                        continue
                     
            elif k.lower().split()[0] in token.lower():
                
                    try:
                        h = k.split()
#                         print(token)
#                         print(len(token.split()))
                        def index(arr, num):
                            a = []
                            for i, x in enumerate(arr):
                                if x == num:
                                    a.append(i)
                            return(a)

                     
                            
                            
                        lis1 = index(token.lower().split(),k.lower().split()[0])
                        lis2 =  index(token.lower().split(),k.lower().split()[1])
                        x = []
                        for i in lis1:
                            for j in lis2:
                                if abs(i - j) == 1:
                                    x.append(i)
                                    
                        num = x[0]  
                        z[num] = 'B'
                        p= len(h)-1
                        y=0
                        for j in range(1,len(token.split())):
                            if (k.lower().split()[j] in token.lower().split()):
#                                 print('rahul')
                                print(j)
                                z[num + j] = 'I'
                                print(k.lower().split()[j])
                                y+=1
                                if y==p:
                                  break
                            else:
                                continue
                            
                    except:
                        continue      
        
        for m, n in enumerate(z):
            if z[m] == 'I' and z[m-1] == 'O':
                z[m] = 'O'

        if set(z) != {'O'}:
            labels.append(z) 
            key_sent.append(token)
    return key_sent, labels

In [6]:
sentences_ = []
labels_ = []
for key, value in filekey.items():
    s, l = convert(key)
    sentences_.append(s)
    labels_.append(l)
sentences = [item for sublist in sentences_ for item in sublist]
labels = [item for sublist in labels_ for item in sublist]
# print(len(sentences), len(labels))

## Model

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [8]:
MAX_LEN = 75
bs = 32

In [9]:
tag2idx = {'B': 0, 'I': 1, 'O': 2}
tags_vals = ['B', 'I', 'O']

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [12]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [13]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [14]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [15]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [16]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [17]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [18]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [19]:
model = model.cuda()

In [20]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [21]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
epochs = 4
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.09159750645318307


Epoch:  25%|██▌       | 1/4 [03:56<11:50, 236.93s/it]

Validation loss: 0.07012036180606594
Validation Accuracy: 0.9785305212620027
F1-Score: 0.2601214574898785
Train loss: 0.06027003754823169


Epoch:  50%|█████     | 2/4 [07:55<07:54, 237.36s/it]

Validation loss: 0.05512299305862851
Validation Accuracy: 0.9826303155006857
F1-Score: 0.4358124111795358
Train loss: 0.04443128131453163


Epoch:  75%|███████▌  | 3/4 [11:53<03:57, 237.66s/it]

Validation loss: 0.04628589225036127
Validation Accuracy: 0.9847713763145862
F1-Score: 0.5485906604964241
Train loss: 0.03500022632185339


Epoch: 100%|██████████| 4/4 [15:51<00:00, 237.85s/it]

Validation loss: 0.04753524796278388
Validation Accuracy: 0.9848725422953818
F1-Score: 0.5591739475774424


.

In [23]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Validation loss: 0.04753524796278388
Validation Accuracy: 0.9848725422953818
Validation F1-Score: 0.5591739475774424


### Get keywords from sentence

In [24]:
def keywordextract(sentence):
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None,
                                  attention_mask=segments_tensors)
    logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

In [27]:
text = "The solution is based upon an abstract representation of the mobile object system."

In [28]:
keywordextract(text)

mobile 0
